In [ ]:
!pip install LogUniform

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import re
from collections import Counter
import time

import loguniform
from loguniform import LogUniform

import numpy as np

from __future__ import print_function
import collections
import math
import numpy as np
import random
import zipfile
import os
import requests

from urllib.request import urlretrieve
from os.path import isfile, isdir
import zipfile
import tensorflow as tf
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
import pandas as pd

from numpy import genfromtxt
import sys

numbBooks = 2829853+1 #according to https://github.com/zygmuntz/goodbooks-10k there are 10000 books in the dataset



In [ ]:

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                

In [ ]:
# download_file_from_google_drive('1w3uSNtc1srNaWoRHwbUE9o3baokloh9U', 'books.csv')

In [ ]:
# dl_id = input("Enter Gdrive file ID for books: ") 

# dl_id =  '1w3uSNtc1srNaWoRHwbUE9o3baokloh9U'

# # DOWNLOAD ZIP
# print ("Downloading  file")
# myzip = drive.CreateFile({'id': dl_id})
# myzip.GetContentFile('books.csv')

# print( os.listdir() )


In [ ]:
# b = pd.read_csv( 'books.csv' )
# b.head(30)
# bookDictionary = b.set_index('book_id').to_dict()['original_title']
# bookDictionary[5]

In [ ]:
download_file_from_google_drive('1lH-0JqQ4UwgL-k83w9xrfftHzvx54phy', 'GoodReadsUser4MS.h5')

In [ ]:
openHdf = pd.read_hdf('GoodReadsUser4MS.h5')

In [ ]:
openHdf.loc[180:220]

,UserID,EmbedID
180,1,180
181,1,181
182,1,182
183,1,183
184,1,184
185,1,185
186,1,186
187,1,187
188,1,188
189,1,189


In [ ]:
my_data = openHdf.groupby('UserID')['EmbedID'].apply(list).values

In [ ]:
my_data

array([list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]),
       list([102, 200, 201, 202, 92, 203, 204, 205, 113, 206, 207, 208, 209, 210, 211, 212, 147

In [ ]:
del openHdf

In [ ]:
data_index = 0
epoch_index = 0
# recEpoch_indexA = 0 #Used to help keep store of the total number of epoches with the models

d1 = LogUniform(a=1, b=numbBooks)

def generate_batch(batch_size, posRate, negRate): 
  
    global data_index, epoch_index
    
    batch = np.zeros(shape=(batch_size, posRate), dtype=np.int32) 
    labels = np.zeros(shape=(batch_size, 1), dtype=np.int32)
    negs = np.zeros(shape=(batch_size, negRate), dtype=np.int32)

    n=0
    while n < batch_size:
        firstpick = np.random.choice( len(my_data[data_index]), 1)[0]
        labels[n] = my_data[data_index][firstpick]
        batchSet = my_data[data_index][:firstpick]+my_data[data_index][firstpick+1:]
        batch[n] = np.random.choice( batchSet , posRate)
        excludes = np.concatenate( (labels[n], batch[n]), axis=None)

        criterea0 = 0
        while criterea0 < negRate: #just in case 
            aa = d1.rvs(negRate*3).astype(int) 
            # aa = d1.rvs(negRate*4).astype(int) -1
            sampleCandidates = np.setdiff1d( aa, excludes )
            criterea0 = sampleCandidates.shape[0]

        negs[n] = np.random.choice( sampleCandidates, negRate, replace=False)

        n = n+1
        data_index = (data_index + 1) % len(my_data) #may have to do something like len my_data[:]
        if data_index == 0:
            epoch_index = epoch_index + 1
            print('Completed %d Epochs' % epoch_index)
    
    return batch, labels, negs    


In [ ]:
here, goes, negs = generate_batch(20, 2, 3) 
print('batch', here)
print('labels', goes)
print('negs', negs)

batch [[  91  131]
 [ 147  235]
 [ 238  238]
 [ 261  124]
 [ 338  313]
 [ 458  459]
 [ 467  472]
 [ 529  491]
 [ 563  579]
 [ 627  620]
 [ 651  656]
 [ 662  662]
 [ 688  716]
 [ 143  736]
 [ 773  745]
 [ 776  776]
 [ 873  833]
 [ 917  911]
 [ 944  941]
 [1001  967]]
labels [[ 49]
 [210]
 [260]
 [262]
 [345]
 [454]
 [474]
 [519]
 [575]
 [619]
 [653]
 [661]
 [680]
 [210]
 [ 72]
 [777]
 [842]
 [914]
 [949]
 [956]]
negs [[     58  882631   30456]
 [   7622  866625   53365]
 [    177 1070178   22946]
 [1199920  452648      89]
 [ 218939       8    2914]
 [     33     541   59866]
 [     15     541    2735]
 [    277 1670320      46]
 [   1170  311616    1844]
 [      1      11    1205]
 [   8390   72898       7]
 [    198       2    1796]
 [ 262039  520751  418196]
 [ 240385 2580133     234]
 [      2     416     348]
 [      4  185514 1771730]
 [     64       4 2023280]
 [ 131648  723777     129]
 [  42188       2      19]
 [ 427173  182536      21]]


In [ ]:
class SkipGramModel(nn.Module):
    """Skip gram model of word2vec.
    Attributes:
        emb_size: Embedding size.
        emb_dimention: Embedding dimention, typically from 50 to 500.
        u_embedding: Embedding for center word.
        v_embedding: Embedding for neibor words.
    """

    def __init__(self, batchSize, skipWindow):
        """Initialize model parameters.
        Apply for two embedding layers.
        Initialize layer weight
        Args:
            emb_size: Embedding size.
            emb_dimention: Embedding dimention, typically from 50 to 500.
        Returns:
            None
        """
        super(SkipGramModel, self).__init__()
        self.emb_size = batchSize
        self.skipWindow = skipWindow
        self.emb_dimension = emb_dimension
        self.batchSize = batchSize
        self.u_embeddings = nn.Embedding(batchSize, emb_dimension, sparse=True).cuda()
        self.v_embeddings = nn.Embedding( batchSize, emb_dimension, sparse=True).cuda()
        # self.targets = torch.ones(34816).cud
        self.init_emb()

    def init_emb(self):
        """Initialize embedding weight like word2vec.
        The u_embedding is a uniform distribution in [-0.5/em_size, 0.5/emb_size], and the elements of v_embedding are zeroes.
        Returns:
            None
        """
        initrange = (2.0 / (numbBooks + self.emb_dimension)) ** 0.5 # Xavier init
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.normal_(mean=0, std=math.sqrt(initrange))
        self.lossFunction = nn.BCEWithLogitsLoss( reduction = 'none' )

    def forward(self, pos_u, pos_v, neg_v, targets ):
        """Forward process.
        As pytorch designed, all variables must be batch format, so all input of this method is a list of word id.
        Args:
            pos_u: list of center word ids for positive word pairs.
            pos_v: list of neibor word ids for positive word pairs.
            neg_u: list of center word ids for negative word pairs.
            neg_v: list of neibor word ids for negative word pairs.
        Returns:
            Loss of this process, a pytorch variable.
        """
        emb_u = self.u_embeddings(pos_u)
        emb_v = self.v_embeddings(pos_v)
        neg_emb_v = self.v_embeddings(neg_v)

        scorePos = torch.bmm(emb_u, torch.transpose(emb_v, 1, 2)).squeeze()
        scoreNeg = torch.bmm( emb_u , torch.transpose(neg_emb_v, 1, 2) ).squeeze()

        #reduce to dot products for each set, and concatinate all the losses
        #noticed that the sign change for the negative sample dot products
        totalScores = torch.cat( ( scorePos.reshape(scorePos.shape[0], -1) , -scoreNeg.reshape(scoreNeg.shape[0], -1) ) , dim=1)

        indLoss = self.lossFunction( totalScores, targets )
        rowSum = torch.sum(indLoss, dim=1) #Sum all losses for each set
        finalLoss = torch.mean(rowSum) #Average losses across batches 

        return finalLoss

In [ ]:
output_file_name = 'theOutpule.file'

emb_dimension = 400
batch_size = 256
posRate = 4
negRate = 128
iterationsMax = 40000 #200001
# initial_lr = 1.0

skip_gram_model = SkipGramModel(batch_size*(2*posRate+negRate), emb_dimension)
use_cuda = torch.cuda.is_available()
print('use cuda ? ', use_cuda)
if use_cuda:
    skip_gram_model.cuda()

optimizer = optim.Adagrad(
    skip_gram_model.parameters())

use cuda ?  True


In [ ]:
import gc

In [ ]:
gc.collect()

207

In [ ]:
!pip install SpeedTorch

In [ ]:
import SpeedTorch

In [ ]:
uEmbed_switcher = SpeedTorch.ModelFactory( skip_gram_model.u_embeddings, total_classes=numbBooks, embed_dimension=400 )
vEmbed_switcher = SpeedTorch.ModelFactory( skip_gram_model.v_embeddings, total_classes=numbBooks, embed_dimension=400 )

In [ ]:
initrange = (2.0 / (numbBooks + emb_dimension)) ** 0.5 # Xavier init
uEmbed_switcher.uniformDistributionInit( -initrange, initrange )
vEmbed_switcher.normalDistributionInit( 0, math.sqrt(initrange) )

In [ ]:
uAdagrad_switcher = SpeedTorch.OptimizerFactory( optimizer, total_classes=numbBooks, embed_dimension=400, model=skip_gram_model, variable_name= 'u_embeddings', CPUPinn = True)
vAdagrad_switcher = SpeedTorch.OptimizerFactory( optimizer, total_classes=numbBooks, embed_dimension=400, model=skip_gram_model, variable_name='v_embeddings' , CPUPinn = True)

In [ ]:
uAdagrad_switcher.optInit()
vAdagrad_switcher.optInit()

In [ ]:

labelsDummyInput = uEmbed_switcher.variableTransformer( batchSize=batch_size, posPerBatch=1)
batchDummpyInput, negzDummyInput = vEmbed_switcher.variableTransformer( batchSize=batch_size, posPerBatch=4, negPerBatch=128 )

In [ ]:
runningLoss = 0
runningStepTime = 0 
negSamp = 128
window_size = posRate

targets = torch.ones( batch_size, window_size + negSamp , dtype = torch.float32 ).cuda()

batchTensor = torch.from_numpy(batchDummpyInput)
LabelTensor = torch.from_numpy(labelsDummyInput)
negTensor = torch.from_numpy(negzDummyInput)

pos_u = Variable(torch.LongTensor(LabelTensor.long()))
pos_v = Variable(torch.LongTensor(batchTensor.long()))
neg_v = Variable(torch.LongTensor(negTensor.long()))

if use_cuda:
    pos_u = pos_u.cuda()
    pos_v = pos_v.cuda()
    neg_v = neg_v.cuda()

for i in range(1000000):
    start = time.time()
    batch, labels, negz = generate_batch(batch_size=batch_size, posRate=4, negRate= 128)

    with torch.no_grad():
        uEmbed_switcher.beforeForwardPass( labels )
        vEmbed_switcher.beforeForwardPass( batch, negz )
        uAdagrad_switcher.beforeForwardPass( labels )
        vAdagrad_switcher.beforeForwardPass( batch, negz )

    optimizer.zero_grad()

    loss = skip_gram_model.forward(pos_u, pos_v, neg_v, targets)
    runningLoss = runningLoss + loss.data.item()

    loss.backward()
    optimizer.step()

    with torch.no_grad():
        uEmbed_switcher.afterOptimizerStep( labels )
        vEmbed_switcher.afterOptimizerStep( batch, negz )
        uAdagrad_switcher.afterOptimizerStep( labels )
        vAdagrad_switcher.afterOptimizerStep( batch, negz )

    end = time.time()
    runningStepTime = runningStepTime + end - start

    if i%1000 == 0:
        print('i is ', i)
        print('loss is ', runningLoss/1000)
        print('Average step time is ', runningStepTime/1000)
        runningLoss = 0
        runningStepTime = 0





i is  52400
loss is  25.44021614074707
Average step time is  0.07218986511230469
i is  52500
loss is  24.706112003326417
Average step time is  0.07175094604492188
i is  52600
loss is  24.208006343841554
Average step time is  0.07194416284561157
i is  52700
loss is  24.235839786529542
Average step time is  0.07187815189361572
i is  52800
loss is  24.177165851593017
Average step time is  0.07203875064849853
i is  52900
loss is  24.288210258483886
Average step time is  0.07186048984527588
i is  53000
loss is  24.202629585266113
Average step time is  0.07172263860702514
i is  53100
loss is  24.121510677337646
Average step time is  0.07126603364944457
i is  53200
loss is  24.20358549118042
Average step time is  0.07123564720153809
i is  53300
loss is  24.186213817596435
Average step time is  0.07180657148361207
i is  53400
loss is  23.910485401153565
Average step time is  0.07178303480148315
i is  53500
loss is  24.145247116088868
Average step time is  0.07169868707656861
i is  53600
loss i

In [ ]:
# i is  98500
# loss is  23.02226722717285
# Average step time is  0.07153946399688721
# Completed 18 Epochs
# i is  98600

In [ ]:
del uAdagrad_switcher
del vAdagrad_switcher

In [ ]:
del my_data

In [ ]:
del vEmbed_switcher

In [ ]:
del skip_gram_model

In [ ]:
gc.collect()

70

In [ ]:
numpyU = uEmbed_switcher.getNumpyVersion()

In [ ]:
del uEmbed_switcher

In [ ]:
# uEmbed_switcher.saveCupy( 'uEmbedsFinal.npy.cpy')
# # vEmbed_switcher.saveCupy( 'vEmbedsFinal.npy.cpy')

In [ ]:
# np.save('testFile.npy',numpyU )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
gc.collect()

In [ ]:
np.save('/content/drive/My Drive/GoodReadsData/Lit2vec2p8Mil2.npy', numpyU )

In [ ]:
!pip install -U -q PyDrive

from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [ ]:
uploadModel = drive.CreateFile() 
uploadModel.SetContentFile('Lit2vec2p8Mil2.npy')
uploadModel.Upload()

In [ ]:
# num_points = 400

# tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
# two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])
# two_d_embeddingsSM = tsne.fit_transform(final_embeddingsSM[1:num_points+1, :])

In [ ]:
# def plot(embeddings, labels):
#   assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
#   pylab.figure(figsize=(50,50))  # in inches
#   for i, label in enumerate(labels):
#     x, y = embeddings[i,:]
#     pylab.scatter(x, y)
#     pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
#                    ha='right', va='bottom')
#   pylab.show()

# books = [bookDictionary[i] for i in range(1, num_points+1)]
# plot(two_d_embeddings, books)

In [ ]:
# def plot(softmax_weightsPlot, labels):
#   assert softmax_weightsPlot.shape[0] >= len(labels), 'More labels than embeddings'
#   pylab.figure(figsize=(50,50))  # in inches
#   for i, label in enumerate(labels):
#     x, y = softmax_weightsPlot[i,:]
#     pylab.scatter(x, y)
#     pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
#                    ha='right', va='bottom')
#   pylab.show()

# books = [bookDictionary[i] for i in range(1, num_points+1)]
# plot(two_d_embeddingsSM, books)